# Importing libraries

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc

# Dataset

This specific ds separates flowers into 3 different classes of species
- setosa
- versicolor
- virginica
The information about each flower is the following
- sepal length
- sepal width
- petal length
- petal width

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
train_path = tf.keras.utils.get_file('iris_training.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')
test_path = tf.keras.utils.get_file('iris_test.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')
#here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [5]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header=0)


In [6]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [7]:
train_y = train.pop('Species')
test_y = test.pop ('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [8]:
train.shape

(120, 4)

In [9]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

# Input Function

In [10]:
def input_fn (features, labels, training=True, batch_size=256):
    #convert the inputs to a dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    #shuffle and repeat if you are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

In [11]:
train.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

# Feature Columns

In [12]:
#feature columns describe how to use the input.
#crea una lista de columnas de caracteristicas que se pueden utilizar como entrada para un modelo en TensorFlow
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the model


We have two options
- DNNClassifier (deep neural network)
- LinearClassifier

DNN seems to be the best choice, this is because we may not be able to find a linear

In [14]:

classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    #two hidden layers of 30 and 10 nodes respectively ---> they are the middle layers in the neural network
    hidden_units = [30,10],
    #the model must choose between 3 classes
    n_classes = 3
)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.

INFO:tensorflow:Using default config.

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\VICTUS\\AppData\\Local\\Temp\\tmp2t9bslfv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker',

# Training

In [20]:
classifier.train(
    input_fn=lambda: input_fn (train, train_y, training=True),
    steps=5000
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\VICTUS\AppData\Local\Temp\tmp2t9bslfv\model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\VICTUS\AppData\Local\Temp\tmp2t9bslfv\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.8196893, step = 5000
INFO:tensorflow:global_step/sec: 817.555
INFO:tensorflow:loss = 0.8180875, step = 5100 (0.125 sec)
INFO:tensorflow:global_step/sec: 1286.72
INFO:tensorflow:loss = 0.80377984, step = 5200 (0.076 sec)
INFO:tensorflow:global_step/sec: 1209.28
INFO:tensorflow:loss = 0.786485, step =

In [21]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn (test, test_y, training=False))
print ('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-07-31T19:50:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\VICTUS\AppData\Local\Temp\tmp2t9bslfv\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18663s
INFO:tensorflow:Finished evaluation at 2024-07-31-19:50:30
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8333333, average_loss = 0.5652116, global_step = 10000, loss = 0.5652116
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\VICTUS\AppData\Local\Temp\tmp2t9bslfv\model.ckpt-10000

Test set accuracy: 0.833



# Predictions


In [22]:
#funcion que crea un dataset de tensorflow
def input_fn (features, batch_size=256):
    #convert the inputs to a Dataset without labels
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

#definicion de las caracteristicas y preparacion del diccionario predict
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}


#solicitud de entrada al usuario
print("Please type numeric values as prompted")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ': ')
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]

#generacion de predicciones
predictions = classifier.predict(input_fn=lambda:input_fn(predict))

#iteracion sobre las predicciones
#toma cada prediccion realizada por el modelo, identifica la clase a la que pertenece esa prediccion 
#y calcula la probabilidad de que esa prediccion sea correcta
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

Please type numeric values as prompted
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\VICTUS\AppData\Local\Temp\tmp2t9bslfv\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (87.8%)


In [23]:
#Here ir some example input and expected classes to try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1]

    #la 1er columna seria setosa, la 2da versicolor y la 3era virginica
}